In [ ]:
from ipyleaflet import *
import ipywidgets
from ipywidgets.embed import embed_minimal_html
from ipywebrtc import WidgetStream, ImageRecorder

import json
import numpy as np
import matplotlib.pyplot as matPlt
#import gdal

import rasterio
from rasterio import plot as rastPlt
from rasterio._base import gdal_version
from rasterio.merge import merge as rasterMerge
from rasterio.mask import mask as rasterMask
from rasterio.io import MemoryFile
from rasterio.crs import CRS
from shapely.geometry import Polygon
import contextily

import ee
from geemap import *

In [ ]:
# To run once for setting up earth engine
#ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
folderName = "img/Test2/"
fig, axs = matPlt.subplots(1,4,figsize=(15,15))

axs[0].set_title("Sentinel-2")
with rasterio.open(folderName + "Sentinel-2_1.tif") as r:
    rastPlt.show(r, ax=axs[0])

axs[1].set_xlim(axs[0].get_xlim())
axs[1].set_ylim(axs[0].get_ylim())
axs[2].set_xlim(axs[0].get_xlim())
axs[2].set_ylim(axs[0].get_ylim())
axs[3].set_xlim(axs[0].get_xlim())
axs[3].set_ylim(axs[0].get_ylim())

axs[1].set_title("Landsat-8")
with rasterio.open(folderName + "Landsat-8_1.tif") as r: 
    rastPlt.show(r, ax=axs[1])

axs[2].set_title("VIIRS Nighttime")
with rasterio.open(folderName + "Night VIIRS_1.tif") as r:
    rastPlt.show(r, ax=axs[2])

axs[3].set_title("Human settlement")
with rasterio.open(folderName + "Population GHSL_1.tif") as r:
    rastPlt.show(r, ax=axs[3])


In [ ]:
def getMosaicFromFiles(fileRaster,meta):
  mosaic, output = rasterMerge(fileRaster)
  meta.update({
      "driver": "GTiff",
      "height": mosaic.shape[1],
      "width": mosaic.shape[2],
      "transform": output,
  })
  return mosaic, meta

def getCoordForPixel(mask, transform):
  dataX, dataY = np.where(mask)
  data = np.c_[dataX, dataY]
  sampleData = []
  for x, y in data:
    p = []
    for offset in ['ul','ur','lr','ll']:
      p.append(rasterio.transform.xy(transform, x, y, offset=offset))
    sampleData.append(p)
  return sampleData

def getImgFromCoord(raster, areas, crop=True):
  pol = []
  for a in areas:
    pol.append(Polygon(a))
  tile, transform = rasterMask(raster, pol, crop=crop)
  return tile, transform

def getTrainingAndTestPerimeter(path, threashold):
  with rasterio.open(path) as file:
    night = file.read(1)
    threashold = 200
    train = getCoordForPixel(night > threashold, file.transform)
    test = getCoordForPixel((0 < night) & (night <= threashold), file.transform)
  return train, test

In [ ]:
folderName = "img/Test2/"
fig, axs = matPlt.subplots(1,2,figsize=(10,10))

train, test = getTrainingAndTestPerimeter(folderName + "Night VIIRS_1.tif", 200)

with rasterio.open(folderName + "Sentinel-2_1.tif") as s2:
  s2Train, transformTrain = getImgFromCoord(s2, train, False)
  s2Test, transformTest = getImgFromCoord(s2, test, False)

  rastPlt.show(s2Train, transform=transformTrain, ax=axs[0])
  rastPlt.show(s2Test, transform=transformTest, ax=axs[1])

# Combine images

In [ ]:
listRaster = []
for i in range(1, 65):
    listRaster.append(rasterio.open('img/Sokoto/Sentinel-2_' + str(i) + '.tif'))
    
mosaic, output = rasterMerge(listRaster)
output_meta = rasterio.open('img/Sokoto/Sentinel-2_1.tif').meta.copy()
output_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": output,
})
#with rasterio.open('img/Sokoto/Sentinel-2.tif', 'w', **output_meta) as r:
#    r.write(mosaic)

In [ ]:
rastPlt.show(mosaic)

# Overlap

In [ ]:
m = geemap.Map(center=(0, 0), zoom=12)
m.add_control(ScaleControl(position='topright'))
m.add_control(LayersControl())

imgPath = os.path.join(os.getcwd(),'img/Sokoto/test_Sentinel-2.tif')
m.add_local_tile(imgPath, bands=[4,3,2],layer_name='pop')

display(m)